In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn

from utils import *

c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
all_files=[]
file_name=[]
import os
for root, dirs, files in os.walk("../data/"):
    for file in files:
        if file.endswith(".csv"):
             all_files.append(os.path.join(root, file))
             file_name.append(f'{root}/{file}')
# datasets with anomalies loading
list_of_df = [pd.read_csv(file, 
                          sep=';', 
                          index_col='datetime', 
                          parse_dates=True) for file in all_files if 'anomaly-free' not in file]

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from utils import *
device = get_default_device()

class Encoder(nn.Module):
  def __init__(self, in_size, latent_size):
    super().__init__()
    
    self.linear1 = nn.Linear(in_size, int(in_size/2))
    self.linear2 = nn.Linear(int(in_size/2), int(in_size/4))
    self.linear3 = nn.Linear(int(in_size/4), latent_size)
    self.relu = nn.ReLU(True)
        
  def forward(self, w):
    out = self.linear1(w)
    out = self.relu(out)
    out = self.linear2(out)
    out = self.relu(out)
    out = self.linear3(out)
    z = self.relu(out)

    return z
    
class Decoder(nn.Module):
  def __init__(self, latent_size, out_size):
    super().__init__()

    self.linear1 = nn.Linear(latent_size, int(out_size/4))
    self.linear2 = nn.Linear(int(out_size/4), int(out_size/2))
    self.linear3 = nn.Linear(int(out_size/2), out_size)
    self.relu = nn.ReLU(True)
    self.sigmoid = nn.Sigmoid()

        
  def forward(self, z):

    out = self.linear1(z)
    out = self.relu(out)
    out = self.linear2(out)
    out = self.relu(out)
    out = self.linear3(out)
    w = self.sigmoid(out)
    return w
    
class UsadModel(nn.Module):
  def __init__(self, w_size, z_size):
    super().__init__()
    self.encoder = Encoder(w_size, z_size)
    self.decoder1 = Decoder(z_size, w_size)
    self.decoder2 = Decoder(z_size, w_size)
  
  def training_step(self, batch, n):
    z = self.encoder(batch)
    w1 = self.decoder1(z)
    w2 = self.decoder2(z)
    w3 = self.decoder2(self.encoder(w1))
    loss1 = 1/n*torch.mean((batch-w1)**2)+(1-1/n)*torch.mean((batch-w3)**2)
    loss2 = 1/n*torch.mean((batch-w2)**2)-(1-1/n)*torch.mean((batch-w3)**2)
    return loss1,loss2

  def validation_step(self, batch, n):
    z = self.encoder(batch)
    w1 = self.decoder1(z)
    w2 = self.decoder2(z)
    w3 = self.decoder2(self.encoder(w1))
    loss1 = 1/n*torch.mean((batch-w1)**2)+(1-1/n)*torch.mean((batch-w3)**2)
    loss2 = 1/n*torch.mean((batch-w2)**2)-(1-1/n)*torch.mean((batch-w3)**2)
    return {'val_loss1': loss1, 'val_loss2': loss2}
        
  def validation_epoch_end(self, outputs):
    batch_losses1 = [x['val_loss1'] for x in outputs]
    epoch_loss1 = torch.stack(batch_losses1).mean()
    batch_losses2 = [x['val_loss2'] for x in outputs]
    epoch_loss2 = torch.stack(batch_losses2).mean()
    return {'val_loss1': epoch_loss1.item(), 'val_loss2': epoch_loss2.item()}
    
  def epoch_end(self, epoch, result):
    #print("Epoch [{}], val_loss1: {:.4f}, val_loss2: {:.4f}".format(epoch, result['val_loss1'], result['val_loss2']))
    pollo=0
    
def evaluate(model, val_loader, n):
    outputs = [model.validation_step(to_device(batch,device), n) for [batch] in val_loader]
    return model.validation_epoch_end(outputs)

def training(epochs, model, train_loader, val_loader, opt_func=torch.optim.Adam):
    history = []


    optimizer1 = opt_func(list(model.encoder.parameters())+list(model.decoder1.parameters()))
    optimizer2 = opt_func(list(model.encoder.parameters())+list(model.decoder2.parameters()))
    for epoch in range(epochs):
        for [batch] in train_loader:
            batch=to_device(batch,device)
            
            #Train AE1
            loss1,loss2 = model.training_step(batch,epoch+1)
            loss1.backward()
            optimizer1.step()
            optimizer1.zero_grad()
            
            
            #Train AE2
            loss1,loss2 = model.training_step(batch,epoch+1)
            loss2.backward()
            optimizer2.step()
            optimizer2.zero_grad()
            
            
        result = evaluate(model, val_loader, epoch+1)
        model.epoch_end(epoch, result)
        history.append(result)
    return history
    
def testing(model, test_loader, alpha=.5, beta=.5):
    results=[]
    for [batch] in test_loader:
        batch=to_device(batch,device)
        w1=model.decoder1(model.encoder(batch))
        w2=model.decoder2(model.encoder(w1))
        results.append(alpha*torch.mean((batch-w1)**2,axis=1)+beta*torch.mean((batch-w2)**2,axis=1))
    return results

In [5]:
#Redefine Window 3-6-12-24-48

WINDOW_SIZE =48
def create_sequences(values, time_steps=WINDOW_SIZE):
    output = []
    for i in range(len(values) - time_steps):
        output.append(values[i : (i + time_steps)])
    return np.stack(output)

In [18]:
from sklearn import preprocessing
import torch.utils.data as data_utils
from tqdm import tqdm
errore=[]
PREDICTION =[]

BATCH_SIZE =  64
N_EPOCHS = 100
hidden_size = 8

iter=1
for df in tqdm(list_of_df):

            anomaly=df['anomaly']
            X_train = df[:400].drop(['anomaly','changepoint'], axis=1)
            X_test = df.drop(['anomaly','changepoint'], axis=1)
            X_val = df[:400].drop(['anomaly','changepoint'], axis=1)
            #X_val = df[:450].drop(['anomaly','changepoint'], axis=1)


            scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled  = scaler.transform(X_test)
            X_val_scaled  = scaler.transform(X_val)

            
            X_train_scaled=create_sequences(X_train_scaled,WINDOW_SIZE)
            X_test_scaled=create_sequences(X_test_scaled,WINDOW_SIZE)
            X_val_scaled=create_sequences(X_val_scaled,WINDOW_SIZE)

            w_size=X_train_scaled.shape[1]*X_train_scaled.shape[2]
            z_size=hidden_size

            train_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
                    torch.from_numpy(X_train_scaled).float().reshape(([X_train_scaled.shape[0],w_size]))
                ) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

            val_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
                    torch.from_numpy(X_val_scaled).float().reshape(([X_val_scaled.shape[0],w_size]))
                ) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

            test_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
                    torch.from_numpy(X_test_scaled).float().reshape(([X_test_scaled.shape[0],w_size]))
                ) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

            model = UsadModel(w_size, z_size)
            model = to_device(model,device)

            #SAVE MODEL
            #history = training(N_EPOCHS,model,train_loader,val_loader)
            #torch.save(model.state_dict(), f'./../MODEL_Saved/USAD/MODEL{WINDOW_SIZE}/model{iter}'')
            
            #LOAD MODEL
            model = UsadModel(w_size, z_size)
            model.load_state_dict(torch.load(f'./../MODEL_Saved/USAD/MODEL{WINDOW_SIZE}/model{iter}'))
            model.eval()

            results=testing(model,test_loader)
            y_pred=np.concatenate([torch.stack(results[:-1]).flatten().detach().cpu().numpy(),
                                results[-1].flatten().detach().cpu().numpy()])
            
            results=testing(model,val_loader)
            y_val=np.concatenate([torch.stack(results[:-1]).flatten().detach().cpu().numpy(),
                                results[-1].flatten().detach().cpu().numpy()])
            

            
            
            th=np.max(y_val)

            outlier=y_pred>th
            
            predi=[False for j in range(len(anomaly))]
            for i in range(len(y_pred)):
                predi[i+WINDOW_SIZE-1]=outlier[i]

            PREDICTION.append(pd.Series(np.array(predi).astype(int), index=df.index))
            iter=iter+1
            
            

100%|██████████| 33/33 [06:00<00:00, 10.93s/it]


In [20]:
import metrics as metric
#EVALUATION METRICS
true_outlier = [df.anomaly for df in list_of_df]
metric.evalu(true_outlier,PREDICTION, metric='binary', window_time='30 sec')

Total value: 36621
TRUE POSITIVE: 12002
TRUE NEGATIVE: 17141
FALSE POSITIVE: 6797
FALSE NEGATIVE: 681
PRECISION: 0.6384382147986595
RECALL: 0.9463060790033904
ACCURANCY: 0.7958002239152399
False Alarm Rate 28.39 %
Missing Alarm Rate 5.37 %
F1 metric 0.7624674417127246


In [11]:
from sklearn import preprocessing
import torch.utils.data as data_utils
from tqdm import tqdm

BATCH_SIZE =  64
N_EPOCHS = 200
hidden_size = 8


import csv
header=['th_factor','method','F1','precision','recall','TP','TN','FP','FN']

mt=['IQR','MAD','STD']

with open('USAD_.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    writer.writerow(header)
    thresholding_factor=[0.5,1,1.5,2]
    for k in range(3):
        for t in thresholding_factor:
            iter=1
            TP=0
            TN=0
            FN=0
            FP=0
            for df in tqdm(list_of_df):

                        anomaly=df['anomaly']
                        X_train = df[:400].drop(['anomaly','changepoint'], axis=1)
                        X_test = df.drop(['anomaly','changepoint'], axis=1)
                        X_val = df[:450].drop(['anomaly','changepoint'], axis=1)
                        


                        scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
                        X_train_scaled = scaler.fit_transform(X_train)
                        X_test_scaled  = scaler.transform(X_test)
                        X_val_scaled  = scaler.transform(X_val)

                        
                        X_train_scaled=create_sequences(X_train_scaled,WINDOW_SIZE)
                        X_test_scaled=create_sequences(X_test_scaled,WINDOW_SIZE)
                        X_val_scaled=create_sequences(X_val_scaled,WINDOW_SIZE)

                        w_size=X_train_scaled.shape[1]*X_train_scaled.shape[2]
                        z_size=hidden_size

                        train_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
                                torch.from_numpy(X_train_scaled).float().reshape(([X_train_scaled.shape[0],w_size]))
                            ) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

                        val_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
                                torch.from_numpy(X_val_scaled).float().reshape(([X_val_scaled.shape[0],w_size]))
                            ) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

                        test_loader = torch.utils.data.DataLoader(data_utils.TensorDataset(
                                torch.from_numpy(X_test_scaled).float().reshape(([X_test_scaled.shape[0],w_size]))
                            ) , batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

                        model = UsadModel(w_size, z_size)
                        model.load_state_dict(torch.load(f'./../MODEL_Saved/USAD/MODEL{WINDOW_SIZE}/model{iter}'))
                        model.eval()


                        results=testing(model,test_loader)
                        y_pred=np.concatenate([torch.stack(results[:-1]).flatten().detach().cpu().numpy(),
                                            results[-1].flatten().detach().cpu().numpy()])
                        
                        results=testing(model,val_loader)
                        y_val=np.concatenate([torch.stack(results[:-1]).flatten().detach().cpu().numpy(),
                                            results[-1].flatten().detach().cpu().numpy()])

                        score=y_val

                        if(k==0):
                            q1, q3 = np.percentile(score, 25), np.percentile(score, 75)
                            iqr = q3 - q1
                            th = q3 + t* iqr
                        if(k==1):
                            median = np.median(score)
                            mad = 1.4826 * np.median(np.abs(score - median))
                            th = median + t * mad

                        if(k==2):
                            mean, std = np.mean(score), np.std(score)
                            th = mean + t * std


                        score=y_pred
                        score=score>th
                        true_= df['anomaly'][WINDOW_SIZE:]>=1
                        prediction_ = score[:].astype(int)==1

                        TP = TP+(true_ & prediction_).sum()   
                        TN = TN+(~true_ & ~prediction_).sum()  
                        FP = FP+(~true_ & prediction_).sum()    
                        FN = FN+(true_ & ~prediction_).sum()    
                        iter=iter+1
                      
            PREC=TP / (TP + FP)
            REC = TP/ (TP+FN)
            f1=2 * PREC * REC/(PREC + REC)
            m=['IQR','MAD','STD']
            row=[t,mt[k],f1,PREC,REC,TP,TN,FP,FN]
            writer.writerow(row)




                        

100%|██████████| 33/33 [00:02<00:00, 11.67it/s]
